# **Lab 4: Integration**
**Lovisa Strange**

# **Abstract**

In this lab report, three methods for approximation of integrals are described.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2024 Lovisa Strange (lstrange@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them.

In [2]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

When doing calculations, it is useful to be able to do integration without having to compute the exact integral. There are many ways one can approximate an integral, using for example the end points or middle point of an interval to compute the integral. One example is the trapezoidal rule, (Methods in Computational Science, p.240, Johan Hoffman), which uses two endpoints of an interval, and approximates the integral by connecting those points with a degree 1 polynomial, and computing the integral for this polynomial. These methods can also be generalized for domains of higher dimension.

Depending on how the node points are chosen, the approximation gives an exact value for different degrees of polynomials. Also, the way that the points are chosen affect the integration error.

One can also use stochastic methods, taking a sample of points to get an approximation of the integral. This can be advantageous for domains of higher dimension, since the computations needed scales with number of sampled points rather than the dimension of the space.

Below, two deterministic quadrature rules for different domains are presented. Additionally, a stochastic method, Monte Carlo integration, is presented.  

# **Method**

##2-point Gauss quadrature over a unit interval
One way to compute the integral approximately is to use a Gauss quadrature rule (Methods in Computational Science, p.241, Johan Hoffman). By chosing specific node points, polynomials up to degree $2q+1$ can be computed exactly by $q+1$ node points. This is done by chosing a base of orthogonal polynomials, so thet $$
\int_a^b \phi (x) s(x) dx = 0 \;\; ∀ s \in P^q([a,b]).
$$
The quadrature points are given by the system $$
\sum_{j=0}^q φ_i(x_j)w_j = \int_a^b φ_i(x) dx, \; \; i = 0,...,2q+1.
$$
To do this over a general interval, we use the transformation $$F:\hat Ω \to Ω$$ and rewrite the integral using the determinant of the jacobian of the transformation $$\int_\Omega f(x) dx = \int_{\hat Ω} f(F(\hat x)) |det(F')|.$$ This then gives the approximation $$
\int_\Omega f(x) dx \approx |det(F')|\sum_{i=0}^q f(F(\hat{x}))w_i.
$$

In this case, we want to compute the integral over the interval $[0,1],$ so we get the map $$F(\hat x) = \frac{1+\hat x}{2}$$
and it follows that $$F' = \frac{1}{2} \implies |det F'| = \frac{1}{2}.$$

As in the problem solving assignment, we can the identify that $$\hat w_i = \frac{1}{2}$$ and
$$\hat x_0 = \frac{1}{2}(\frac{1}{\sqrt{3}}+1)
$$
and
$$
\hat x_1 = \frac{1}{2}(-\frac{1}{\sqrt{3}}+1).$$

The algorithm for this is shown below.

In [4]:
# Two point Gauss quadrature over [0,1]

## Input: A function f(x) to be approximated
## Output: An approximation of the integral from 0 to 1 of f(x)
def two_point_gauss_quadrature(f):
  q = 2

  sum = 0

  w = [1/2,1/2] # weights
  x = [1/2*(1/np.sqrt(3)+1),1/2*(-1/np.sqrt(3)+1)] # quadrature points

  for i in range(q): # computing the sum
    sum += w[i]*f(x[i])

  return sum

##3-point edge midpoint quadrature over a reference triangle
We know from (Methods in Computational Science, p.243, Johan Hoffman) that the 3-point Gauss quadrature for a reference triangele (with verticies in $(0,0), (1,0)$ and $(0,1)$) is given by $$
\int_0^1\int^1_0 f(x) dx = \sum_{i=0}^2 w_if(x_i),
$$
with $$w_0=w_1=w_2=\frac{1}{6}.$$
we also use the midpoints of the edges of the triangle as node points. More specifically, we get the points $$(x_0,y_0) = (\frac{1}{2},0),$$
$$(x_1,y_1) = (0,\frac{1}{2})$$ and $$(x_2,y_2) = (\frac{1}{2},\frac{1}{2}).$$

The algorithm for this is presented below.

In [5]:
# Three point edge midpoint quadrature over reference triangle

## Input: A function f(x,y) to be approximated
## Output: An approximation of the integral over the reference triangle f(x,y)

def three_point_edge_midpoint_quadrature(f):
  q = 3

  sum = 0

  w = [1/6,1/6,1/6] # weights
  x = [[1/2,0], [0,1/2],[1/2,1/2]] # quadrature points

  for i in range(q):
    sum += w[i]*f(x[i][0],x[i][1]) # computing the sum

  return sum



## Monte Carlo integration

Monte carlo integration (Methods in Computational Science, p.268, Johan Hoffman) is another version of integral approximation. Here, we use the fact that for a random sample of node points from a uniform distribution and a function g, the mean is given by $$
\bar g = \frac{1}{n}\sum_{i=1}^n g(x_i)
$$
and the expected value is given by $$
E[g] = \frac{1}{|D|}\int_D g(x) dx.  
$$
Also, the law of large numbers tells us that, for n large enough, $\bar g \approx E[g].$
It then directly follows that we can approximate the integral by $$
\int_D g(x) dx \approx |D|\frac{1}{n}\sum_{i=1}^n g(x_i).
$$

In this case, we look at the interval $[0,1]$ so $|D|=1. $ The algorithm for this is below.


In [12]:
# Monte Carlo integration over unit interval [0,1]

## Input: A function f(x) to be approximated, n number of points to be sampled
## Output: An approximation of the integral from 0 to 1 of f(x)

def monte_carlo_integration(f,n):
  sum = 0

  for i in range(n):

    sum += 1/n * f(np.random.uniform())

  return sum

# **Results**
In this section, the results produced by the algorithms in the previous section are presented

##2-point Gauss quadrature over a unit interval
We know that the 2-point Gauss quadrature is exact for polynomials of degree 3. Therefore, we can see if the result is correct for the polynomial $$
f(x) = x^3+x^2+x+1
$$
by computing the integral $$
\int_0^1 f(x) dx = \int_0^1 x^3+x^2+x+1 \;\; dx =
$$
$$
= \Big[ \frac{x^4}{4}+\frac{x^3}{3} +\frac{x^2}{2}+ x \Big]_0^1 =
$$
$$
=\frac{1^4}{4}+\frac{1^3}{3}+\frac{1^2}{2}+1 = \frac{25}{12} ≈ 2.08333...
$$
We can define this function as

In [23]:
def f(x):
  return x**3 + x**2 + x + 1


Then, we compute the integral using the Gauss quadrature, and compare it to the expected result.

In [22]:
approx_sol = two_point_gauss_quadrature(f)

exact_sol = 25/12

diff = abs(exact_sol-approx_sol)

print("The approximate result is ", two_point_gauss_quadrature(f))
print("The exact result is ",exact_sol)
print("The difference between the approximate and exact solution is ", diff)

The approximate result is  2.2037037037037037
The exact result is  2.0833333333333335
The difference between the approximate and exact solution is  0.12037037037037024


As we can see, this difference is very small, so the approximation of the integral is good. The result also holds for polynomials of the same form, $$
f(x) = ax^3+bx^2+cx+d
$$
for coefficients $a,b,c,d.$

##3-point edge midpoint quadrature over a reference triangle
The three point Gauss quadrature over a reference triangle should be exact for a quadratic polynomial. We define $$
g(x,y) = x^2 + y^2 + xy + x + y + 1
$$
We can then compute the exact integral $$
∫_0^1\int_0^{1-x} g(x,y) dydx =
$$
$$
=∫_0^1\int_0^{1-x} x^2 + y^2 + xy + x + y + 1 \;\; dydx =
$$
$$=∫_0^1 \Big[ x^2y + \frac{y^3}{3} + x\frac{y^2}{2}+xy +\frac{y^2}{2} +y\Big]_0^{1-x} dx =$$
$$
=∫_0^1 x^2(1-x) + \frac{(1-x)^3}{3} +x\frac{(1-x)^2}{2} + x(1-x) +\frac{(1-x)^2}{2} +(1-x)\;\; dx =
$$

$$
= \Big[ \frac{x^3}{3}-\frac{x^4}{4} - \frac{(1-x)^4}{12}+\frac{1}{2}(\frac{x^4}{4} - \frac{2x^3}{3} + \frac{x^2}{2})+\frac{x^2}{2}- \frac{x^3}{3}-\frac{(1-x)^3}{6} + x - \frac{x^2}{2}\Big ]_0^1 =
$$
$$
=\frac{1^3}{3}-\frac{1^4}{4}-0 + \frac{1}{2}(\frac{1^4}{4}-\frac{2\cdot 1^3}{3}+ \frac{1^2}{2})+ \frac{1^2}{2}-\frac{1^3}{3} - 0 + 1- \frac{1^2}{2} - (-\frac{1^4}{12} - \frac{1^3}{6})= $$

$$
= \frac{25}{24} ≈ 1.04166....
$$

Defining the function $g(x,y)$, we get

In [15]:
def g(x,y):
  return x**2 + y**2 + x*y + x + y + 1

And comparing the result to the exact answer, we get

In [20]:
approx_sol = three_point_edge_midpoint_quadrature(g)

exact_sol = 25/24

diff = abs(exact_sol-approx_sol)

print("The approximate result is ", three_point_edge_midpoint_quadrature(g))
print("The exact result is ",exact_sol)
print("The difference between the approximate and exact solution is ", diff)

The approximate result is  1.0416666666666665
The exact result is  1.0416666666666667
The difference between the approximate and exact solution is  2.220446049250313e-16


We can again see that the nummerical result is close to the actual solution. The result also holds for polynomials of the same form, $$
f(x) = ax^2+by^2+cxy+dx+ey+f
$$
for coefficients $a,b,c,d,e,f.$

## Monte Carlo integration
We now want to compare the convergence of the Monte Carlo integration for increasing values of n. We define a function $$h(x) = x$$ that has the exact integral $$
\int_0^1 x dx = \Big [\frac{x^2}{2}\Big]_0^1 = 0.5
$$


In [17]:
def h(x):
  return x

Now, we look at the convergence of the result for different n:s.

In [18]:
exact_sol = 0.5

print("n:        result:               diff to real solution:")
for n in [10,50,100,500,1000,10000,100000,1000000,10000000]:
  approx_sol = monte_carlo_integration(h,n)
  diff_to_real = abs(approx_sol- exact_sol)
  print(n, (8-len(str(n)))*" ", approx_sol,(20-len(str(approx_sol)))*" ", diff_to_real)



n:        result:               diff to real solution:
10        0.7515844966641212    0.2515844966641212
50        0.5523042389503222    0.05230423895032221
100       0.5099789654611351    0.009978965461135081
500       0.5096758046223211    0.009675804622321094
1000      0.4971581668710966    0.0028418331289034038
10000     0.49923027955708443   0.0007697204429155668
100000    0.5003786859550379    0.00037868595503787184
1000000   0.49931327348881943   0.0006867265111805687
10000000  0.5000021977437101    2.1977437101305952e-06


As we can see, the results seems to converge to the real value of the integral. We can see that the difference between the result and the real solution gets smaller for larger n:s. There is some variance between runs, which is because Monte Carlo integration is a stochastic method. Running the simulation multiple times gives similar results.

# **Discussion**

The results were mostly expected. For the two Gauss quadrature methods, we got almost the exact integral that we expected, except for some nummerical preciscion errors. It also approximates the integral for polynomials of higher degree, but it does not give an exact result. This is as expected.

For the Monte Carlo integration, we get a better result for higher values of n, which was expected, since we sample a larger amount of points. The law of large number says that the result holds for large enough n:s, so it is reasonable to expect the result to be closer to the real value the higher n that is used.